In [30]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [103]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display
from six.moves import cPickle as pickle
import string
from PIL import Image
import numpy as np
import h5py

In [104]:
width = None
pd.options.display.max_rows = 600
pd.options.display.max_columns = width
pd.options.display.max_colwidth = 600
pd.options.display.width = width
pd.options.display.max_seq_items = None
pd.options.display.expand_frame_repr = False
pd.options.display.colheader_justify = 'left'

In [105]:
import data_commons as dtc
import dl_commons as dlc

In [125]:
class VisualizeDir(object):
    def __init__(self, storedir, gen_datadir='../data/generated2'):
        self._storedir = storedir
        self._logdir = os.path.join(storedir, '..')
        self._hyper = dtc.load(self._logdir, 'hyper.pkl')
        self._args = dtc.load(self._logdir, 'args.pkl')

        self._word2id = pd.read_pickle(os.path.join(gen_datadir, 'dict_vocab.pkl'))
        i2w = pd.read_pickle(os.path.join(gen_datadir, 'dict_id2word.pkl'))
        for i in range(-1,-11,-1):
            i2w[i] = '%d'%i
        self._id2word = {}
        ## Append space after all commands beginning with a backslash (except backslash alone)
        for i, w in i2w.items():
            if w[0] == '\\':
              self._id2word[i] = w + " "  
            else:
                self._id2word[i] = w 
        self._id2word[self._word2id['id']['\\']] = '\\'
    
    @property
    def storedir(self):
        return self._storedir
    
    @property
    def w2i(self):
        return self._word2id['id']

    @property
    def i2w(self):
        return self._id2word
    
    @property
    def max_steps(self):
        steps = [int(os.path.basename(f).split('_')[-1].split('.')[0]) for f in os.listdir(self._storedir)]
        epoch_steps = [int(os.path.basename(f).split('_')[-1].split('.')[0]) for f in os.listdir(self._storedir) if f.startswith('validation')]
        return sorted(steps)[-1], sorted(epoch_steps)[-1]
        
    @property
    def args(self):
        return self._args
    
    @property
    def hyper(self):
        return self._hyper
    
    def keys(self, graph, step):
        with h5py.File(os.path.join(self._storedir, '%s_%d.h5'%(graph,step))) as h5:
            return h5.keys()

    def np(self, graph, step, key):
        """
        Args:
            graph: 'training' or 'validation'
            step:  step who's output is to be fetched
            key:   key of object to fetch - e.g. 'predicted_ids'
        """
        with h5py.File(os.path.join(self._storedir, '%s_%d.h5'%(graph,step))) as h5:
            return h5[key][...]
    
    def df(self, graph, step, key):
        return pd.DataFrame(self.np(graph, step, key))
    
    def words(self, graph, step, key, key2=None):
        df = self.df(graph, step, key)
        df2 = self.df(graph, step, key2) if (key2 is not None) else None
        
        if key2 is None:
            return df.applymap(lambda x: self._id2word[x])
        else:
            return pd.DataFrame({'%s'%key: df.applymap(lambda x: self._id2word[x]), '%s'%key2: df2.applymap(lambda x: self._id2word[x])})

    def strs(self, graph, step, key, key2=None, mingle=True):
        df_str = self.words(graph, step, key)
        df_str2 = self.words(graph, step, key2) if (key2 is not None) else None
        
        ## each token's string version - excepting backslash - has a space appended to it,
        ## therefore the string output should be compile if the prediction was syntactically correct
        if key2 == None:
            return pd.DataFrame(["".join(row) for row in df_str.itertuples(index=False)])
        else:
            if mingle:
                ar1 = ["".join(row) for row in df_str.itertuples(index=False)]
                ar2 = ["".join(row) for row in df_str2.itertuples(index=False)]
                data = {'%s_%d %s / %s\t\t(%s)'%(graph, step, key, key2, self._storedir): [e for t in zip(ar1, ar2) for e in t]}
            else:
                data = {'%s_%d.%s\t\t(%s)'%(graph, step, key, self._storedir): ["".join(row) for row in df_str.itertuples(index=False)], '%s_%d.%s\t\t(%s)'%(graph, step, key2, self._storedir): ["".join(row) for row in df_str2.itertuples(index=False)]}

            df = pd.DataFrame(data)
#             df.style.set_caption('%s/%s_%s'%(self._storedir, graph, step))
            return df
        
    def prune_logs(self, save_epochs=1):
        """Save the latest save_epochs logs and remove the rest."""
        epoch_steps = [int(os.path.basename(f).split('_')[-1].split('.')[0]) for f in os.listdir(self._storedir) if f.startswith('validation')]
        if len(epoch_steps) <= save_epochs:
            print('Only %d full epochs were found. Deleting nothing.'%epoch_steps)
            return False
        else:
            epoch_steps.sort(reverse=True)
            max_step = epoch_steps[save_epochs]
            training_steps = [int(os.path.basename(f).split('_')[-1].split('.')[0]) for f in os.listdir(self._storedir) if f.startswith('training')]
            steps_to_remove = filter(lambda s: (s<max_step) and (s not in epoch_steps), training_steps)
            files_to_remove = [f for f in os.listdir(self._storedir) if f.startswith('training') and (int(os.path.basename(f).split('_')[-1].split('.')[0]) in steps_to_remove) ]
            print 'The following files will be removed', files_to_remove

class VisualizeStep():
    def __init__(self, visualizer, step):
        self._step = step
        self._visualizer = visualizer
        
    def keys(self, graph):
        return self._visualizer.keys(graph, self._step)
    
    def np(self, graph, key):
        return self._visualizer.np(graph, self._step, key)
    
    def df(self, graph, step, key):
        return pd.DataFrame.df(self.np(graph, step, key))
    
    def words(self, graph, key, key2=None):
        return self._visualizer.words(graph, self._step, key, key2)

    def strs(self, graph, key, key2=None, mingle=True):
        return self._visualizer.strs(graph, self._step, key, key2, mingle)

class DiffParams(object):
    def __init__(self, dir1, dir2):
        self._dir1 = dir1
        self._dir2 = dir2
        
    def get(self, filename, to_str):
        one = dtc.load(self._dir1, filename)
        two = dtc.load(self._dir2, filename)
        if (to_str):
            one = dlc.to_dict(one)
            two = dlc.to_dict(two)
        return one, two

    def print_dict(self, filename, to_str):
        one, two = self.get(filename, to_str)
        dtc.pprint(dlc.diff_dict(one, two))
    
    def _table(self, filename):
        one, two = self.get(filename, False)
        head, tail = dlc.diff_table(one, two)
        display(pd.DataFrame(head))
        display(pd.DataFrame(tail))
        
    def args(self, to_str=True):
        self._table('args.pkl')        
        
    def hyper(self, to_str=True):
        self._table('hyper.pkl')
    
    def get_args(self):
        return self.get('args.pkl', to_str=True)
    def get_hyper(self):
        return self.get('hyper.pkl', to_str=True)

In [126]:
# v = Visualize('./tb_metrics_dev/2017-10-06 17-56-47 PDT/store', '../data/generated2')
# v = VisualizeDir('./tb_metrics/2017-10-08 12-26-45 PDT/store')
# v = VisualizeDir('./tb_metrics_dev/2017-10-09 12-45-15 PDT/store')
# vd = VisualizeDir('./tb_metrics/2017-10-09 17-43-49 PDT/store')
vd = VisualizeDir('tb_metrics/2017-09-26 22-40-18 PDT/my_decoder 2017-10-10 16-03-26 PDT/store')
vd2 = VisualizeDir('tb_metrics/2017-09-26 22-40-18 PDT/new_code 2017-10-10 15-10-17 PDT/store')

In [131]:
display(vd.max_steps)
display(vd2.max_steps)

(89096, 89096)

(93350, 93240)

In [132]:
vs = VisualizeStep(vd2, 93240)
vs.strs('validation', 'predicted_ids', 'y', mingle=False)

,validation_93240.predicted_ids (tb_metrics/2017-09-26 22-40-18 PDT/new_code 2017-10-10 15-10-17 PDT/store),validation_93240.y (tb_metrics/2017-09-26 22-40-18 PDT/new_code 2017-10-10 15-10-17 PDT/store)
0,"g_{ij}\>\\equiv \8\mbox {erd}\\\\G=\,\,\,\,\,\,\,\,\,\,\,\,{\rm eyd}\,\,\,\,\,\,g=\eta _{ik}\eos \eos \eos \eos \eos \eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2","g_{ij}\;=\;\Pi _i^{\hat {r}}\Pi ^{\hat {s}}_j\,\eta _{\hat {r}\hat {s}}\,,\quad g\;=\;{\rm det}g_{ij}\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
1,"\tilde {F}_\epsilon (X)=(X^0)^{-1/2}B_B(X)\,+\,c_g^{(1)}\,\bar {\xi }_g^{(t)}\,\bar {P}_g^{-1}(F)+(X^2)\,\lambda \,.\label {21}\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2","\tilde {F}_g(X)=(X^0)^{2-2g}\tilde {F}_g(Z)=(g_s^2)^{g-1}e^{(1-g)K}\tilde {F}_g(Z)\,.\label {Fg}\eos \eos \eos \eos \eos \eos \eos \eos \eos"
2,H^{\mu \nu }_H=\partial ^{\mu }B^{\nu }_\sigma +\partial ^{\nu }B^{\sigma }_1+\partial ^{\sigma }E^{\nu }_R=2ig^{\mu \nu }_F~.\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2,{\overline H}_R^{\mu \nu \sigma }=\partial ^{\mu }{\overline C}^{\nu \sigma }_R+\partial ^{\nu }{\overline C}^{\sigma \mu }_R+\partial ^{\sigma }{\overline C}^{\mu \nu }_R~.\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
3,"X=q^{-{H\over 2}}X^-r^{-{H\over 2}}X^-,\quad \tilde {q}=q^{-R},\quad \bar {q}^2=q^{-{D\over 2}}X^-r^{-2}r^{\frac {d}{2}}r^{-2}\eos \eos \eos \eos \eos \eos \eos \eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2","X=q^{-{H\over 2}}X_-,\quad \bar X=q^{-{H\over 2}}X_+,\quad g=q^{-H}q^{C},\quad \bar g=q^{-H}q^{{-C}}\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
4,"MF=\frac {O}{P_0}\,\,L=\rm \mbox {Aod}\,\,\,,~,~~~\,\,\,I_2\in \left (\begin {array}{cc}0&I\\I&0\label {errap}\right )\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2","\label {rest}M'LM'=-L,\quad {\rm where}\quad L=\left (\begin {array}{cc}0&I\\I&0\end {array}\right )\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
5,"x_\mu ^L(z)\psi -J^{(\mu ,c)}\psi (x)-u^{\mu ,\,a}(x),\,\,\,\,\,g^\mu _L(z,z)\tilde u^\nu =-\frac {L_\mu ^*\,d}{x-w}\,{,}\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2","x_L^I(z)x_L^J(w)\sim -\eta ^{IJ}\log (z-w),~~\psi _L^I(z)\psi _L^J(w)\sim -{\eta ^{IJ}\over z-w}\,\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
6,"\delta _R\,=\,e^{\beta +4}\,(4-4){\rm tr}\,e^{-\frac {k}{4}}\,\Lambda ^{-\frac {1}{2}}\,\partial _{i}^{(\frac {1}{2})}\,\partial _{i}(\vec {k}\,+\/\partial _{i}\vec {j}\,).\label {t3}\eos -1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2",\delta _FA=\bar ss/2[(e^1+e^2)A\bar A+(e^3+e^4A\bar A)(N+\bar N)].\label {dFA1}\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
7,"\label {stran}~(P_i^\prime ,S_i)=2P_{i+}\otimes \Omega _j,\\\D_1\otimes S_{i-1}\equiv 0.\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2","\label {6.4}\{\underline Q_{\underline \alpha },\underline Q_{\underline \beta }\}=2iP_{\underline a}(C\Gamma ^{\underline a})_{\underline \alpha \underline \beta },\quad [\underline Q_{\underline \alpha },P_{\underline a}]=0.\eos \eos \eos \eos \eos \eos \eos \eos \eos

In [43]:
# diff = DiffParams('./tb_metrics/2017-10-07 14-33-35 PDT', './tb_metrics_dev/2017-10-09 12-45-15 PDT')
# diff = DiffParams('./tb_metrics/2017-09-26 22-40-18 PDT', './tb_metrics/2017-10-07 14-33-35 PDT')
# diff = DiffParams('./tb_metrics/2017-10-07 14-33-35 PDT', './tb_metrics/2017-10-08 12-26-45 PDT')
diff = DiffParams('./tb_metrics/2017-09-26 22-40-18 PDT', './tb_metrics/2017-10-08 12-26-45 PDT')
# diff = DiffParams('./tb_metrics/2017-09-26 22-40-18 PDT/w=1', './tb_metrics/2017-10-08 12-26-45 PDT')
# diff = DiffParams('./tb_metrics/2017-09-26 22-40-18 PDT', './tb_metrics_dev/2017-10-09 12-45-15 PDT')

In [ ]:
diff.get_hyper()[0]['CALSTM_STACK']

In [ ]:
diff.args()

In [ ]:
diff.get_hyper()[1]['squash_input_seq']

In [68]:
v.w2i['\\']

61